# 忘年会席決め計算

## 問題

$n$ 人を $n$ 個の座席に配置する. 
このとき以下のように配置する. 

- 同一プロジェクトに配属されている人はできるだけ違うテーブルに配置
- 同一グループに所属している人はできるだけ違うテーブルに配置
- 同年代はできるだけ違うテーブルに配置(社員の年齢が確認できる場合)
- その他同属性の社員はできるだけ違うテーブルに配置

## 定式化

座席ごとではなくテーブル単位で割り当てる

### パラメータ

- $S_t$: テーブル $t$ の座席集合
- $E_p$: プロジェクト $p$ に配属されているメンバー
- $E_g$: グループ $g$ に配属されているメンバー
- $E_a$: 年代 $a$ に属しているメンバー

### 変数

- $x_{it}$: 人 $i$ をテーブル $t$ に配置するとき $1$, そうでないとき $0$

### 制約

- $\sum_i x_{it} \leq |S_t|$ (for all $t$): 1 つのテーブルに座れるのはテーブルのキャパシティ以下だけ
- $\sum_t x_{it} = 1$ (for all $i$): 1 人の人は 1 つのテーブルにだけ割り当てられる

### 目的関数

以下のコスト関数を重み付けして足す. 

- 同一プロジェクト同一テーブル配置ペナルティ
  - $\sum_p \sum_t \left( \sum_{i \in E_p} x_{i t} \right)^2$
- 同一グループ同一テーブル配置ペナルティ
  - $\sum_g \sum_t \left( \sum_{i \in E_g} x_{i t} \right)^2$
- 同一世代同一テーブル配置ペナルティ
  - $\sum_a \sum_t \left( \sum_{i \in E_a} x_{i t} \right)^2$

2 乗になっている部分を展開して 0-1 変数の席の部分を線形制約で表現すれば線形目的関数にすることができる. 
また, 近い位置にあるテーブルに同属性の社員を配置したくない場合は 2 乗の部分を単一テーブルではなく複数のテーブルも対象にすればよい. 

## テスト実装

In [1]:
from ortools.sat.python import cp_model
from ortools.math_opt.python import mathopt
import random
import datetime

### データ生成

In [2]:
class Employee:
    def __init__(self, id, projects, group, age, name=""):
        self.id = id
        self.group = group.id
        self.projects = [p.id for p in  projects]
        self.age = age.id
        self.name = name

        for p in projects:
            p.members.append(self.id)
        group.members.append(self.id)
        age.members.append(self.id)

class Project:
    def __init__(self, id, name=""):
        self.id = id
        self.members = []
        self.name = name

class Group:
    def __init__(self, id, name=""):
        self.id = id
        self.members = []
        self.name = name

class Age:
    def __init__(self, id, name=""):
        self.id = id
        self.members = []
        self.name=name

class Sheet:
    def __init__(self, id, table, name=""):
        self.id = id
        self.table = table.id
        self.name = name

        table.sheets.append(self.id)

class Table:
    def __init__(self, id, name=""):
        self.id = id
        self.sheets = []
        self.name = name

    @property
    def capacity(self):
        return len(self.sheets)

In [3]:
# n_all: 人数
# s_per_t: 1 テーブルあたりの席数
n_all = 65
s_per_t = 5

s_all = n_all
t_all = (s_all + s_per_t - 1) // s_per_t

# テーブルは順番に番号付ける
tables = [Table(t_id) for t_id in range(t_all)]
sheets = [Sheet(s_id, tables[s_id // s_per_t]) for s_id in range(s_all)]

# プロジェクト, グループ, 年代を適当に生成
p_all = 5
g_all = 4
projects = [Project(p_id) for p_id in range(p_all)]
groups = [Group(g_id) for g_id in range(g_all)]
ages = [Age(id, name) for id, name in enumerate(["20~30 代", "40~50 代", "60 代以上"])]

random.seed(0)
employees = []
for e_id in range(n_all):
    e_group = random.choice(groups)
    e_projects = random.sample(projects, random.randint(1, len(projects) // 2))  # プロジェクトをランダムに割り当て
    e_age = random.choice(ages)

    employees.append(Employee(e_id, e_projects, e_group, e_age))

In [4]:
for e in employees:
    print(f"Employee {e.id}", end=": ")
    print(f"group={e.group}", end=" ")
    print(f"projects={e.projects}", end=" ")
    print(f"age={e.age}({ages[e.age].name})")

Employee 0: group=3 projects=[0, 2] age=2(60 代以上)
Employee 1: group=3 projects=[2, 3] age=1(40~50 代)
Employee 2: group=1 projects=[2] age=0(20~30 代)
Employee 3: group=0 projects=[4, 1] age=1(40~50 代)
Employee 4: group=0 projects=[2] age=1(40~50 代)
Employee 5: group=0 projects=[3, 2] age=2(60 代以上)
Employee 6: group=1 projects=[3, 2] age=0(20~30 代)
Employee 7: group=0 projects=[3] age=2(60 代以上)
Employee 8: group=0 projects=[2, 1] age=2(60 代以上)
Employee 9: group=2 projects=[1] age=2(60 代以上)
Employee 10: group=1 projects=[1] age=2(60 代以上)
Employee 11: group=3 projects=[0] age=1(40~50 代)
Employee 12: group=3 projects=[2] age=2(60 代以上)
Employee 13: group=2 projects=[4] age=1(40~50 代)
Employee 14: group=1 projects=[3, 0] age=2(60 代以上)
Employee 15: group=3 projects=[4, 1] age=1(40~50 代)
Employee 16: group=1 projects=[1] age=0(20~30 代)
Employee 17: group=2 projects=[0, 4] age=2(60 代以上)
Employee 18: group=1 projects=[0] age=0(20~30 代)
Employee 19: group=3 projects=[4, 1] age=0(20~30 代)
Employee 

In [5]:
for p in projects:
    print(f"Project {p.id}: members={p.members}")

Project 0: members=[0, 11, 14, 17, 18, 27, 29, 31, 32, 33, 43, 46, 49, 51, 52, 54, 56, 58, 61, 62]
Project 1: members=[3, 8, 9, 10, 15, 16, 19, 24, 25, 30, 31, 34, 37, 44, 45, 47, 49, 50, 53, 57, 60]
Project 2: members=[0, 1, 2, 4, 5, 6, 8, 12, 21, 22, 23, 24, 36, 38, 41, 43, 45, 54, 57, 59, 62, 63]
Project 3: members=[1, 5, 6, 7, 14, 20, 28, 34, 40, 41, 46, 48, 50, 52, 63, 64]
Project 4: members=[3, 13, 15, 17, 19, 20, 22, 26, 32, 35, 39, 42, 44, 47, 48, 55, 58]


In [6]:
for g in groups:
    print(f"Group {g.id}: members={g.members}")

Group 0: members=[3, 4, 5, 7, 8, 22, 25, 27, 30, 31, 32, 33, 43, 44, 50, 55, 56, 61]
Group 1: members=[2, 6, 10, 14, 16, 18, 23, 24, 26, 28, 38, 40, 47, 53, 57, 64]
Group 2: members=[9, 13, 17, 21, 29, 34, 39, 42, 45, 46, 52, 60, 62]
Group 3: members=[0, 1, 11, 12, 15, 19, 20, 35, 36, 37, 41, 48, 49, 51, 54, 58, 59, 63]


In [7]:
for a in ages:
    print(f"Age {a.id}: members={a.members}")

Age 0: members=[2, 6, 16, 18, 19, 22, 23, 28, 34, 35, 37, 46, 47, 48, 53, 54, 57, 59, 61]
Age 1: members=[1, 3, 4, 11, 13, 15, 24, 32, 36, 39, 44, 49, 51, 52, 58, 60]
Age 2: members=[0, 5, 7, 8, 9, 10, 12, 14, 17, 20, 21, 25, 26, 27, 29, 30, 31, 33, 38, 40, 41, 42, 43, 45, 50, 55, 56, 62, 63, 64]


In [8]:
for t in tables:
    print(f"Table {t.id}: sheets={t.sheets}")

Table 0: sheets=[0, 1, 2, 3, 4]
Table 1: sheets=[5, 6, 7, 8, 9]
Table 2: sheets=[10, 11, 12, 13, 14]
Table 3: sheets=[15, 16, 17, 18, 19]
Table 4: sheets=[20, 21, 22, 23, 24]
Table 5: sheets=[25, 26, 27, 28, 29]
Table 6: sheets=[30, 31, 32, 33, 34]
Table 7: sheets=[35, 36, 37, 38, 39]
Table 8: sheets=[40, 41, 42, 43, 44]
Table 9: sheets=[45, 46, 47, 48, 49]
Table 10: sheets=[50, 51, 52, 53, 54]
Table 11: sheets=[55, 56, 57, 58, 59]
Table 12: sheets=[60, 61, 62, 63, 64]


### モデル化(CP-SAT)

In [9]:
model = cp_model.CpModel()
x = [[model.new_bool_var(f"e {e.id} -> t {t.id}") for t in tables] for e in employees]

for e in employees:
    model.add_exactly_one([x[e.id][t.id] for t in tables])

for t in tables:
    model.add(sum(x[e.id][t.id] for e in employees) <= t.capacity)

obj_p = 0
obj_g = 0
obj_a = 0

for t in tables:
    for p in projects:
        tp1 = model.new_int_var(0, min(t.capacity, len(p.members)), f"number_of_employee_in_project{p.id}_in_table{t.id}")
        model.add(tp1 == sum(x[e_id][t.id] for e_id in p.members))
        tp2 = model.new_int_var(0, min(t.capacity, len(p.members)) ** 2, "")
        model.add_multiplication_equality(tp2, [tp1, tp1])
        obj_p += tp2

    for g in groups:
        tp1 = model.new_int_var(0, min(t.capacity, len(g.members)), f"number_of_employee_in_group{g.id}_in_table{t.id}")
        model.add(tp1 == sum(x[e_id][t.id] for e_id in g.members))
        tp2 = model.new_int_var(0, min(t.capacity, len(g.members)) ** 2, "")
        model.add_multiplication_equality(tp2, [tp1, tp1])
        obj_g += tp2

    for a in ages:
        tp1 = model.new_int_var(0, min(t.capacity, len(a.members)), f"number_of_employee_in_age{a.id}_in_table{t.id}")
        model.add(tp1 == sum(x[e_id][t.id] for e_id in a.members))
        tp2 = model.new_int_var(0, min(t.capacity, len(a.members)) ** 2, "")
        model.add_multiplication_equality(tp2, [tp1, tp1])
        obj_a += tp2

model.minimize(obj_p + obj_g + obj_a)

In [10]:
solver = cp_model.CpSolver()
status = solver.solve(model)

statuses = {
    cp_model.OPTIMAL: "OPTIMAL",
    cp_model.FEASIBLE: "FEASIBLE",
    cp_model.INFEASIBLE: "INFEASIBLE",
    cp_model.MODEL_INVALID: "MODEL_INVALID",
    cp_model.UNKNOWN: "UNKNOWN",
}

print(f"status = {statuses[status]}")
print(f"time = {solver.wall_time}")
print(f"objective value = {solver.objective_value}")

status = OPTIMAL
time = 0.7439380000000001
objective value = 374.0


In [11]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for t in tables:
        print(f"Table {t.id}:")
        for e in employees:
            if not solver.value(x[e.id][t.id]) == 1:
                continue

            print(f"  employee {e.id}: projects={e.projects} group={e.group} age={e.age}")

Table 0:
  employee 11: projects=[0] group=3 age=1
  employee 20: projects=[3, 4] group=3 age=2
  employee 23: projects=[2] group=1 age=0
  employee 45: projects=[2, 1] group=2 age=2
  employee 61: projects=[0] group=0 age=0
Table 1:
  employee 3: projects=[4, 1] group=0 age=1
  employee 17: projects=[0, 4] group=2 age=2
  employee 18: projects=[0] group=1 age=0
  employee 57: projects=[2, 1] group=1 age=0
  employee 63: projects=[3, 2] group=3 age=2
Table 2:
  employee 2: projects=[2] group=1 age=0
  employee 9: projects=[1] group=2 age=2
  employee 41: projects=[2, 3] group=3 age=2
  employee 44: projects=[1, 4] group=0 age=1
  employee 56: projects=[0] group=0 age=2
Table 3:
  employee 10: projects=[1] group=1 age=2
  employee 15: projects=[4, 1] group=3 age=1
  employee 43: projects=[0, 2] group=0 age=2
  employee 46: projects=[3, 0] group=2 age=0
  employee 59: projects=[2] group=3 age=0
Table 4:
  employee 25: projects=[1] group=0 age=2
  employee 29: projects=[0] group=2 age=2
 

### モデル化(SCIP)

In [12]:
model = mathopt.Model(name="sheet")

x = [[model.add_binary_variable(name=f"e {e.id} -> t {t.id}") for t in tables] for e in employees]

for e in employees:
    model.add_linear_constraint(sum(x[e.id][t.id] for t in tables) == 1)

for t in tables:
    model.add_linear_constraint(sum(x[e.id][t.id] for e in employees) <= t.capacity)

obj_p = 0
obj_g = 0
obj_a = 0

for t in tables:
    for p in projects:
        tp = model.add_integer_variable(lb=0, ub=min(t.capacity, len(p.members)), name=f"number_of_employee_in_project{p.id}_in_table{t.id}")
        model.add_linear_constraint(tp == sum(x[e_id][t.id] for e_id in p.members))
        obj_p += tp * tp

    for g in groups:
        tp = model.add_integer_variable(lb=0, ub=min(t.capacity, len(g.members)), name=f"number_of_employee_in_group{g.id}_in_table{t.id}")
        model.add_linear_constraint(tp == sum(x[e_id][t.id] for e_id in g.members))
        obj_g += tp * tp


    for a in ages:
        tp = model.add_integer_variable(lb=0, ub=min(t.capacity, len(a.members)), name=f"number_of_employee_in_age{a.id}_in_table{t.id}")
        model.add_linear_constraint(tp == sum(x[e_id][t.id] for e_id in a.members))
        obj_a += tp * tp

model.minimize(obj_p + obj_g + obj_a)

In [13]:
params = mathopt.SolveParameters(enable_output=True)
result = mathopt.solve(model, mathopt.SolverType.GSCIP, params=params)

presolving:
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 1 chg bounds, 0 chg sides, 0 chg coeffs, 78 upgd conss, 0 impls, 65 clqs
   (0.1s) probing cycle finished: starting next cycle
   Deactivated symmetry handling methods, since SCIP was built without symmetry detector (SYM=none).
   Deactivated symmetry handling methods, since SCIP was built without symmetry detector (SYM=none).
presolving (2 rounds: 2 fast, 2 medium, 2 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 1 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 3783 implications, 65 cliques
presolved problem has 1002 variables (845 bin, 156 int, 0 impl, 1 cont) and 235 constraints
     13 constraints of type <knapsack>
     65 constraints of type <setppc>
    156 constraints of type <linear>
      1 constraints of type <nonlinear>
Presolving Time: 0.07

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   

In [14]:
if result.termination.reason == mathopt.TerminationReason.OPTIMAL or result.termination.reason == mathopt.TerminationReason.FEASIBLE:
    for t in tables:
        print(f"Table {t.id}:")
        for e in employees:
            if not round(result.variable_values()[x[e.id][t.id]]) == 1:
                continue

            print(f"  employee {e.id}: projects={e.projects} group={e.group} age={e.age}")

Table 0:
  employee 10: projects=[1] group=1 age=2
  employee 22: projects=[4, 2] group=0 age=0
  employee 27: projects=[0] group=0 age=2
  employee 34: projects=[3, 1] group=2 age=0
  employee 36: projects=[2] group=3 age=1
Table 1:
  employee 28: projects=[3] group=1 age=0
  employee 37: projects=[1] group=3 age=0
  employee 42: projects=[4] group=2 age=2
  employee 43: projects=[0, 2] group=0 age=2
  employee 51: projects=[0] group=3 age=1
Table 2:
  employee 26: projects=[4] group=1 age=2
  employee 45: projects=[2, 1] group=2 age=2
  employee 50: projects=[1, 3] group=0 age=2
  employee 54: projects=[2, 0] group=3 age=0
  employee 58: projects=[0, 4] group=3 age=1
Table 3:
  employee 11: projects=[0] group=3 age=1
  employee 39: projects=[4] group=2 age=1
  employee 56: projects=[0] group=0 age=2
  employee 57: projects=[2, 1] group=1 age=0
  employee 63: projects=[3, 2] group=3 age=2
Table 4:
  employee 31: projects=[1, 0] group=0 age=2
  employee 35: projects=[4] group=3 age=0
 

## 線形計画モデルとてしての定式化

### パラメータ

- $S_t$: テーブル $t$ の座席集合
- $E_p$: プロジェクト $p$ に配属されているメンバー
- $E_g$: グループ $g$ に配属されているメンバー
- $E_a$: 年代 $a$ に属しているメンバー

### 変数

- $x_{it}$: 人 $i$ をテーブル $t$ に配置するとき $1$, そうでないとき $0$

### 制約

- $\sum_i x_{it} \leq |S_t|$ (for all $t$): 1 つのテーブルに座れるのはテーブルのキャパシティ以下だけ
- $\sum_t x_{it} = 1$ (for all $i$): 1 人の人は 1 つのテーブルにだけ割り当てられる
- 下記目的関数を表現するための制約

### 目的関数

以下のコスト関数を重み付けして足す. 

- 同一プロジェクト同一テーブル配置ペナルティ
  - $\sum_p \sum_t \left( \sum_{i \in E_p} x_{i t} \right)^2$: これを下記制約により線形化する
      - $\sum_p \sum_t \sum_{i \in E_p} \sum_{j \in E_p} y_{i j t}^{(p)}$
      - $x_{i t} + x_{j t} \leq y_{i j t}^{(p)} + 1$
- 同一グループ同一テーブル配置ペナルティ
  - $\sum_g \sum_t \left( \sum_{i \in E_g} x_{i t} \right)^2$: これを下記制約により線形化する
      - $\sum_p \sum_t \sum_{i \in E_g} \sum_{j \in E_g} y_{i j t}^{(g)}$
      - $x_{i t} + x_{j t} \leq y_{i j t}^{(g)} + 1$
- 同一世代同一テーブル配置ペナルティ
  - $\sum_a \sum_t \left( \sum_{i \in E_a} x_{i t} \right)^2$: これを下記制約により線形化する
      - $\sum_p \sum_t \sum_{i \in E_a} \sum_{j \in E_a} y_{i j t}^{(a)}$
      - $x_{i t} + x_{j t} \leq y_{i j t}^{(a)} + 1$

## テスト実装(線形)

In [15]:
model = mathopt.Model(name="sheet")

x = [[model.add_binary_variable(name=f"e {e.id} -> t {t.id}") for t in tables] for e in employees]

for e in employees:
    model.add_linear_constraint(sum(x[e.id][t.id] for t in tables) == 1)

for t in tables:
    model.add_linear_constraint(sum(x[e.id][t.id] for e in employees) <= t.capacity)

obj_p = 0
obj_g = 0
obj_a = 0

for t in tables:
    for p in projects:
        for e1_id in p.members:
            for e2_id in p.members:
                tp = model.add_binary_variable()
                model.add_linear_constraint(x[e1_id][t.id] + x[e2_id][t.id] <= tp + 1)
                obj_p += tp

    for g in groups:
        for e1_id in g.members:
            for e2_id in g.members:
                tp = model.add_binary_variable()
                model.add_linear_constraint(x[e1_id][t.id] + x[e2_id][t.id] <= tp + 1)
                obj_g += tp

    for a in ages:
        for e1_id in a.members:
            for e2_id in a.members:
                tp = model.add_binary_variable()
                model.add_linear_constraint(x[e1_id][t.id] + x[e2_id][t.id] <= tp + 1)
                obj_a += tp

model.minimize(obj_p + obj_g + obj_a)

### ソルバー比較

どのソルバーも現実的な時間で終わらなかったのでタイムリミットを 5 分に設定

#### 実行: CP-SAT

In [16]:
params = mathopt.SolveParameters(time_limit=datetime.timedelta(minutes=5), enable_output=True)
result = mathopt.solve(model, mathopt.SolverType.CP_SAT, params=params)


Running basic LP presolve, initial problem dimensions: 58058 rows, 58825 columns, 172692 entries with magnitude in [1.000000e+00, 2.000000e+00]
glop::FixedVariablePreprocessor                        58058 rows, 58825 columns, 172692 entries with magnitude in [1.000000e+00, 2.000000e+00]
glop::SingletonPreprocessor                            58058 rows, 58825 columns, 172692 entries with magnitude in [1.000000e+00, 2.000000e+00]
glop::ForcingAndImpliedFreeConstraintPreprocessor      58058 rows, 58825 columns, 172692 entries with magnitude in [1.000000e+00, 2.000000e+00]
glop::FreeConstraintPreprocessor                       58058 rows, 58825 columns, 172692 entries with magnitude in [1.000000e+00, 2.000000e+00]
glop::UnconstrainedVariablePreprocessor                58058 rows, 58825 columns, 172692 entries with magnitude in [1.000000e+00, 2.000000e+00]

Scaling to pure integer problem.
Num integers: 58825/58825 (implied: 0 in_inequalities: 0 max_scaling: 0) [IP] 
Maximum constraint coe

#### 実行: SCIP

In [17]:
params = mathopt.SolveParameters(time_limit=datetime.timedelta(minutes=5), enable_output=True)
result = mathopt.solve(model, mathopt.SolverType.GSCIP, params=params)

presolving:
(round 1, fast)       2938 del vars, 2938 del conss, 0 add conss, 0 chg bounds, 2938 chg sides, 2938 chg coeffs, 0 upgd conss, 0 impls, 65 clqs
(round 2, exhaustive) 2938 del vars, 2938 del conss, 0 add conss, 0 chg bounds, 2938 chg sides, 2938 chg coeffs, 55120 upgd conss, 0 impls, 65 clqs
   (0.2s) probing: 51/55887 (0.1%) - 0 fixings, 0 aggregations, 0 implications, 0 bound changes
   (0.2s) probing aborted: 50/50 successive totally useless probings
   Deactivated symmetry handling methods, since SCIP was built without symmetry detector (SYM=none).
   Deactivated symmetry handling methods, since SCIP was built without symmetry detector (SYM=none).
presolving (3 rounds: 3 fast, 2 medium, 2 exhaustive):
 2938 deleted vars, 2938 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 2938 changed sides, 2938 changed coefficients
 0 implications, 65 cliques
presolved problem has 55887 variables (55887 bin, 0 int, 0 impl, 0 cont) and 55120 constraints
   

#### 実行: Highs

In [18]:
params = mathopt.SolveParameters(time_limit=datetime.timedelta(minutes=5), enable_output=True)
result = mathopt.solve(model, mathopt.SolverType.HIGHS, params=params)

Coefficient ranges:
  Matrix [1e+00, 2e+00]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 5e+00]
Presolving model
58058 rows, 58825 cols, 172692 nonzeros  0s
58058 rows, 58825 cols, 172692 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   58058 rows
   58825 cols (58825 binary, 0 integer, 0 implied int., 0 continuous)
   172692 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.5s
 R       0       0         0   0.00%   226             760               70.26%        0      0      0       223     0.6s
 C       0       0         0   0.00%   226             759               70.22%      614     37      0       789     

### 結果

CP-SAT ソルバーが最良解の目的関数値が最も良く, 378 だった(最適値は 374). 
SCIP と Highs は目的関数値 410 程度までしか得られなかった. 
Dual bound は全てのソルバーで 226 程度だった. 